In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution, dual_annealing
from tqdm import tqdm
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr
import librosa
import os
from IPython.display import Audio, display
from generate_wave_file import generate_wave_file

# Function definitions
def sine_wave(frequency, amplitude, duration, sample_rate, modulator=None):
    time_vector = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    if modulator is not None:
        return amplitude * np.sin(2 * np.pi * frequency * time_vector + modulator)
    return amplitude * np.sin(2 * np.pi * frequency * time_vector)

def fm_modulate(carrier_freq, carrier_amp, modulator_signal, duration, sample_rate):
    time_vector = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    return carrier_amp * np.sin(2 * np.pi * carrier_freq * time_vector + modulator_signal)

def compute_mfcc(signal, sample_rate, n_mfcc=20):
    mfccs = librosa.feature.mfcc(y=signal.astype(np.float32), sr=sample_rate, n_mfcc=n_mfcc)
    return np.mean(mfccs, axis=1)

def compute_objective(params, target_freqs, target_amps, duration, sample_rate, objective_type, target_mfcc_mean=None):
    freq4, amp4, freq3, amp3, freq2, amp2, freq1, amp1 = params
    
    mod4 = sine_wave(freq4, amp4, duration, sample_rate)
    mod3 = fm_modulate(freq3, amp3, mod4, duration, sample_rate)
    mod2 = fm_modulate(freq2, amp2, mod3, duration, sample_rate)
    combined_signal = fm_modulate(freq1, amp1, mod2, duration, sample_rate)
    
    # Normalize combined signal
    combined_signal /= np.max(np.abs(combined_signal))
    
    # Compute FFT
    fft_result = np.abs(np.fft.rfft(combined_signal))
    fft_freqs = np.fft.rfftfreq(len(combined_signal), 1/sample_rate)
    
    # Compute the target spectrum
    target_spectrum = np.zeros_like(fft_result)
    np.add.at(target_spectrum, np.searchsorted(fft_freqs, target_freqs), target_amps)
    
    epsilon = 1e-10  # Small value to avoid log(0) and division by zero
    
    if objective_type == 'itakura_saito':
        return np.sum((target_spectrum + epsilon) / (fft_result + epsilon) - np.log((target_spectrum + epsilon) / (fft_result + epsilon)) - 1)
    elif objective_type == 'spectral_convergence':
        return np.linalg.norm(fft_result - target_spectrum) / np.linalg.norm(target_spectrum)
    elif objective_type == 'cosine_similarity':
        return cosine(fft_result, target_spectrum)
    elif objective_type == 'euclidean_distance':
        return np.linalg.norm(fft_result - target_spectrum)
    elif objective_type == 'manhattan_distance':
        return np.sum(np.abs(fft_result - target_spectrum))
    elif objective_type == 'kullback_leibler_divergence':
        return np.sum(target_spectrum * np.log((target_spectrum + epsilon) / (fft_result + epsilon)))
    elif objective_type == 'pearson_correlation_coefficient':
        return 1 - pearsonr(fft_result, target_spectrum)[0]
    elif objective_type == 'mfcc_distance':
        generated_mfcc_mean = compute_mfcc(combined_signal, sample_rate)
        return np.linalg.norm(generated_mfcc_mean - target_mfcc_mean)

def optimize_audio(frequencies, amplitudes, duration, sample_rate, objective_type='kullback_leibler_divergence', optimization_method='differential_evolution', n_generations=2500):
    target_signal = np.sum([sine_wave(freq, amp, duration, sample_rate) for freq, amp in zip(frequencies, amplitudes)], axis=0)
    target_signal /= np.max(np.abs(target_signal))
    target_mfcc_mean = compute_mfcc(target_signal, sample_rate)

    bounds = [(50, 5000), (0, 12)] * 4  # Frequency and amplitude bounds for 4 oscillators

    pbar = tqdm(total=n_generations, unit=' iteration', bar_format='{l_bar}{bar} {n}/{total} [ETA: {remaining}, Elapsed: {elapsed}]')
    error_history = []

    def callback(param, convergence=None, context=None):
        error = compute_objective(param, frequencies, amplitudes, duration, sample_rate, objective_type, target_mfcc_mean)
        error_history.append(error)
        pbar.update(1)

    if optimization_method == 'differential_evolution':
        result = differential_evolution(lambda params: compute_objective(params, frequencies, amplitudes, duration, sample_rate, objective_type, target_mfcc_mean),
                                        bounds, strategy='best1bin', maxiter=n_generations, popsize=10, tol=1e-6, mutation=(0.5, 1), recombination=0.7, callback=callback)
    elif optimization_method == 'dual_annealing':
        result = dual_annealing(lambda params: compute_objective(params, frequencies, amplitudes, duration, sample_rate, objective_type, target_mfcc_mean),
                                bounds, maxiter=n_generations, callback=lambda x, f, context: callback(x, f))
    else:
        raise ValueError("Invalid optimization method")

    pbar.close()
    return result.x, error_history

def generate_combined_signal(optimal_params, duration, sample_rate):
    optimal_frequencies = optimal_params[0::2]
    optimal_amplitudes = optimal_params[1::2]

    mod4 = sine_wave(optimal_frequencies[0], optimal_amplitudes[0], duration, sample_rate)
    mod3 = fm_modulate(optimal_frequencies[1], optimal_amplitudes[1], mod4, duration, sample_rate)
    mod2 = fm_modulate(optimal_frequencies[2], optimal_amplitudes[2], mod3, duration, sample_rate)
    combined_signal = fm_modulate(optimal_frequencies[3], optimal_amplitudes[3], mod2, duration, sample_rate)

    return combined_signal / np.max(np.abs(combined_signal))

def plot_results(combined_signal, frequencies, amplitudes, error_history, sample_rate):
    # Time domain plot
    plt.figure(figsize=(12, 6))
    plt.plot(combined_signal[:1000])
    plt.title('Optimized Combined Signal in Time Domain')
    plt.xlabel('Sample')
    plt.ylabel('Amplitude')
    plt.show()

    # Frequency domain plot
    fft_result = np.fft.fft(combined_signal)
    fft_freqs = np.fft.fftfreq(len(fft_result), 1/sample_rate)
    fft_result_np = np.abs(fft_result)
    fft_result_np /= np.max(fft_result_np)

    plt.figure(figsize=(12, 6))
    plt.plot(fft_freqs[:len(fft_freqs)//2], fft_result_np[:len(fft_result_np)//2], label='Optimized Spectrum')
    plt.scatter(frequencies, amplitudes, color='red', label='Target Spectrum', zorder=5)
    plt.title('Frequency Spectrum of Optimized Combined Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.show()

    # Error history plot
    plt.figure(figsize=(12, 6))
    plt.plot(error_history)
    plt.title('Error Rate Progression')
    plt.xlabel('Generation')
    plt.ylabel('Distance')
    plt.grid(True)
    plt.show()

def print_optimal_params(optimal_params):
    for i, (freq, amp) in enumerate(zip(optimal_params[0::2], optimal_params[1::2])):
        print(f"Modulator {4-i}:\n")
        print(f"    Frequency: {freq}")
        print(f"    Amplitude: {amp}\n")

# Constants and configurations
DURATION = 1.0  # seconds
SAMPLE_RATE = 44100  # samples per second
FILE_PATH = 'tsv/cello_single.tsv'
OUTPUT_FILENAME = 'optimized_output_fm_cellotest.wav'

# Data loading and preprocessing
df = pd.read_csv(FILE_PATH, sep='\t')
frequencies = df['Frequency (Hz)'].values
amplitudes = df['Amplitude'].values
amplitudes /= np.max(amplitudes)

# Main execution
optimal_params, error_history = optimize_audio(frequencies, amplitudes, DURATION, SAMPLE_RATE, objective_type='kullback_leibler_divergence', optimization_method='differential_evolution', n_generations=1000)
combined_signal = generate_combined_signal(optimal_params, DURATION, SAMPLE_RATE)
plot_results(combined_signal, frequencies, amplitudes, error_history, SAMPLE_RATE)
print_optimal_params(optimal_params)

# Synthesize and save the audio
generate_wave_file(combined_signal, SAMPLE_RATE, custom_filename=OUTPUT_FILENAME, save_to_file=True)

# Display the generated audio file
output_path = os.path.join('rendered_audio', OUTPUT_FILENAME)
display(Audio(filename=output_path))


  0%|                                                                                   0/1000 [ETA: ?, Elapsed: 00:00]

NameError: name 'objective_function' is not defined